In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
# Base url to service
url = 'https://api.hh.ru/'
url_vacancies = 'https://api.hh.ru/vacancies'

In [3]:
# сделала запрос, чтобы получить список всех area для hh
area = 'areas/countries' 
url_area = url + area
r = requests.get(url_area)

# по документации <Response [200]> это успешная обработка
# распарсила этот запрос
area_names = json.loads(r.text)

# пробежалась по каждой area, вытащила в виде списка id для каждой area, какая это area и ее url,
# потом сделала его в виде dataframe
area_names_list = []
index = 0

for i in range(len(area_names)):
    dict_ = {'id' : area_names[i]['id'], 'country' : area_names[i]['name'], 'url' : area_names[i]['url']}
    df_ = pd.DataFrame(dict_, index = [index])
    index += 1
    area_names_list.append(df_)
    
area_names_df = pd.concat(area_names_list)
area_names_df

,id,country,url
0,113,Россия,https://api.hh.ru/areas/113
1,2112,Абхазия,https://api.hh.ru/areas/2112
2,5,Украина,https://api.hh.ru/areas/5
3,6,Австралия,https://api.hh.ru/areas/6
4,40,Казахстан,https://api.hh.ru/areas/40
...,...,...,...
156,211,ЮАР,https://api.hh.ru/areas/211
157,110,Южная Корея,https://api.hh.ru/areas/110
158,2524,Южная Осетия,https://api.hh.ru/areas/2524
159,2440,Ямайка,https://api.hh.ru/areas/2440


In [4]:
# создала параметры, через которые ищу вакансии содержащие слово Аналитик в кз
params = {'text': 'name:Аналитик', 'area': 40}
# сделала запрос 
r = requests.get(url_vacancies, params=params)
# посчитала сколько вообще вакансий получилось
found = json.loads(r.text)['found']; #кол-во всего найденных вакансий
found

331

In [5]:
# всего нашли 300 с лишним вакансий (их число меняется, пока я все это делаю)
# эмпирически я нашла, что можно выгружать не более 100 айдишников за раз, потом надо перелистывать страницу
# поэтому если найдено меньше 100 то получим все сразу.
# поделила количество вакансий из found на количество вакансий на страницу, получается, что на последней странице
# вакансий меньше 100 (числа с плавающей точкой), поэтому сделала округление в большую сторону
# чтобы посчитать количество страниц (страницы будут показывать и меньше 100 вакансий) и перевела в целые числа
# если больше 100 то "перелистываем" страницы с 0 по (наше посчитанное количество) и получаем айди вакансий со всех страниц. 
# говорят, что API не отдаст больше 2000 вакансий (я сама не проверяла)

list_id = []
if found <= 100:
    params['per_page'] = found
    r = requests.get(url_vacancies, params=params)
    data = json.loads(r.text)['items']
    for vac in data:
        list_id.append(vac['id'])
else:
    i = 0;
    while i <= int(np.ceil(found/100)):   
        params['per_page'] = 100
        params['page'] = i
        r = requests.get(url_vacancies, params=params)
        if 200 != r.status_code:
            break
        data = json.loads(r.text)['items']
        for vac in data:
            list_id.append(vac['id'])
        i += 1
        
print('количество полученных айди', len(list_id))

количество полученных айди 331


In [5]:
# list_id

In [3]:
def get_vacancy(id):

    url_vac = 'https://api.hh.ru/vacancies/%s'
    r = requests.get(url_vac % id)

    return json.loads(r.text)

In [6]:
# посмотрела как выглядят json'ы
# get_vacancy(54287190)

In [57]:
df = []
a = 0
for i in list_id:
    vacancy = get_vacancy(i)
    dict_ = {'id' : vacancy['id'], 'name' : vacancy['name'], 'city' : vacancy['area']['name'], 
             'experience' : vacancy['experience']['name'], 'type' : vacancy['type']['name'],
             'address': vacancy['address'], 'published_at' : vacancy['published_at'],
             'archived' : vacancy['archived'], 'employer' : vacancy['employer']['name'],
             'contacts': vacancy['contacts'], 'schedule': vacancy['schedule']['id'], 
             'employer_trusted' : vacancy['employer']['trusted'],'professional_roles' : vacancy['professional_roles'][0]['name']}
    df_ = pd.DataFrame(dict_, index = [a])
    print(a) # были проблемы с ответами, они часто прерывались, поэтому я завела себе счетчик
    a += 1
    df.append(df_)

45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308

In [58]:
df_1 = pd.concat(df)
df_1

,id,name,city,experience,type,address,published_at,archived,employer,contacts,schedule,employer_trusted,professional_roles
45,54506940,Системный аналитик,Алматы,От 3 до 6 лет,Открытая,NaN,2022-04-01T10:02:41+0300,False,Исткомтранс,None,fullDay,True,Аналитик
46,53214719,Финансовый аналитик,Алматы,От 3 до 6 лет,Открытая,NaN,2022-03-31T12:44:11+0300,False,ДБ Альфа-Банк,None,fullDay,True,"Финансовый аналитик, инвестиционный аналитик"
47,52993864,Аналитик по финансовому учету,Алматы,От 3 до 6 лет,Открытая,NaN,2022-03-28T10:49:02+0300,False,Первый БИТ,None,fullDay,True,"Финансовый аналитик, инвестиционный аналитик"
48,54444673,Начинающий Продуктолог / Product manager / Pro...,Алматы,Нет опыта,Открытая,NaN,2022-03-30T08:40:41+0300,False,ASPEX,None,fullDay,True,Менеджер продукта
49,53365939,Аналитик отдела продаж,Алматы,От 1 года до 3 лет,Открытая,NaN,2022-03-29T11:37:21+0300,False,STYNERGY,None,fullDay,True,Аналитик
...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,53263509,IT аналитик,Алматы,От 3 до 6 лет,Открытая,None,2022-03-02T11:22:02+0300,False,PAYBOX Technologies,None,fullDay,True,Аналитик
327,53271269,Администратор Битрикс24 (Бизнес-аналитик Битри...,Алматы,От 1 года до 3 лет,Открытая,None,2022-03-02T13:40:01+0300,False,Орион Система,None,fullDay,True,Аналитик
328,53416630,Химик-аналитик,Алматы,От 1 года до 3 лет,Открытая,None,2022-03-05T12:55:11+0300,False,Snack Distribution,None,shift,True,"Научный специалист, исследователь, лаборант"
329,52825178,"Консультант-аналитик по бухгалтерскому, налого...",Усть-Каменогорск,От 3 до 6 лет,Открытая,None,2022-03-02T11:56:21+0300,False,1С-Рейтинг,None,fullDay,True,Бухгалтер


In [ ]:
# у меня были проблемы с хх, апи не захотело нормально отвечать на запросы
# пришлось данные грузить по частям, создавать пустой датафрейм, и слепливать эти куски, сохраняя индексацию 

In [43]:
vacancy_df_tmp = pd.DataFrame()

In [59]:
vacancy_df = pd.concat([vacancy_df,df_1])

In [65]:
vacancy_df.shape

(331, 13)

In [1]:
vacancy_df.head()

NameError: name 'vacancy_df' is not defined

In [64]:
vacancy_df.to_csv(r'C:\Users\kebek\hh_research\vacancy_df.csv', index = False)

In [ ]:
# а еще я забыла сбросить индексы))))))

In [7]:
# get_vacancy(53613405)